In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
def get_vector(text):
    doc = nlp(text)
    vec = np.array(doc.vector)
    return vec

In [8]:
dataframe = pd.read_csv("resources/bsnewsfilterTrain.csv", header=None)
dataframe = dataframe[1:]
dataframe['Title_vec'] = dataframe.apply(lambda row: get_vector(row[0]),axis=1)
dataframe = dataframe.drop(0,axis=1)
dataset = dataframe.values


In [9]:
X = list(dataframe['Title_vec'])
X = pd.DataFrame(X)
X = X.values[:,0:].astype(float)
Y = dataset[:,0:1].astype(float)

In [ ]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(48, input_dim=96, activation='relu'))
    model.add(Dense(10, input_dim=48, activation='relu'))
    #model.add(Dense(50, input_dim=50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=128, verbose=0)
kfold = StratifiedKFold(n_splits=9, shuffle=True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [13]:
bsfilterModel = create_baseline()
bsfilterModel.fit(X,Y,batch_size=128, epochs=50)

Epoch 1/50
5/5 [==============================] - 0s 2ms/step - loss: 0.7546 - accuracy: 0.5303
Epoch 2/50
5/5 [==============================] - 0s 3ms/step - loss: 0.6724 - accuracy: 0.5825
Epoch 3/50
5/5 [==============================] - 0s 3ms/step - loss: 0.6618 - accuracy: 0.6263
Epoch 4/50
5/5 [==============================] - 0s 3ms/step - loss: 0.6505 - accuracy: 0.6549
Epoch 5/50
5/5 [==============================] - 0s 2ms/step - loss: 0.6339 - accuracy: 0.6599
Epoch 6/50
5/5 [==============================] - 0s 2ms/step - loss: 0.6176 - accuracy: 0.6599
Epoch 7/50
5/5 [==============================] - 0s 3ms/step - loss: 0.6037 - accuracy: 0.6515
Epoch 8/50
5/5 [==============================] - 0s 4ms/step - loss: 0.5935 - accuracy: 0.6549
Epoch 9/50
5/5 [==============================] - 0s 2ms/step - loss: 0.5826 - accuracy: 0.6717
Epoch 10/50
5/5 [==============================] - 0s 3ms/step - loss: 0.5713 - accuracy: 0.6886
Epoch 11/50
5/5 [======================

In [14]:
text = 'George Floyd murder: Apple CEO Tim Cook commits USD 100 million to promote racial equality - The New Indian Express'
def bsfilter(model,text):
    vec = get_vector(text)
    X = pd.DataFrame(vec)
    X = X.values[:,0:].astype(float)
    prediction = model.predict(X.T)[0][0]
    return prediction


In [15]:
bsfilter(bsfilterModel,text)

0.89450395

In [17]:
out = pd.read_csv('out/out 2.csv')
out['bsFilter'] = out.apply(lambda row:bsfilter(bsfilterModel,row['Title']),axis=1)
out = out[out['bsFilter']>=0.2]
#out.to_csv('out/out.csv')
out[out['Event_Text']!='0'][['Event_Text', 'PubDate','HNWI']].to_csv('out/time_line_filtered.csv')